In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import time


class FacialExpressionDetector:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.expression_model = None
        self.img_size = 48
        self.class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        self.expression_colors = {
            'angry': (0, 0, 255),  # Red
            'disgust': (0, 128, 0),  # Dark Green
            'fear': (128, 0, 128),  # Purple
            'happy': (0, 255, 255),  # Yellow
            'neutral': (255, 255, 255),  # White
            'sad': (255, 0, 0),  # Blue
            'surprise': (0, 165, 255)  # Orange
        }
        self.load_model()

    def load_model(self):
        """Load the trained expression model"""
        model_path = 'models/expression_model_final.h5'
        if os.path.exists(model_path):
            try:
                self.expression_model = keras.models.load_model(model_path)
                print("✅ Facial expression model loaded successfully!")
                return True
            except Exception as e:
                print(f"❌ Error loading expression model: {e}")

        print("❌ No trained expression model found.")
        print("💡 Please run train_expression_model.py first")
        return False

    def detect_expressions(self, frame):
        """Detect faces and their expressions"""
        if self.expression_model is None:
            return frame, []

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

        expressions_detected = []

        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = gray[y:y + h, x:x + w]

            # Preprocess for model
            processed_face = self.preprocess_face(face_roi)

            # Predict expression
            expression, confidence = self.predict_expression(processed_face)

            # Get color for expression
            color = self.expression_colors.get(expression, (255, 255, 255))

            # Draw enhanced bounding box
            self.draw_expression_box(frame, x, y, w, h, expression, confidence, color)

            expressions_detected.append({
                'expression': expression,
                'confidence': confidence,
                'bbox': (x, y, w, h),
                'color': color
            })

        return frame, expressions_detected

    def preprocess_face(self, face_img):
        """Preprocess face image for model prediction"""
        # Resize to model input size
        face_img = cv2.resize(face_img, (self.img_size, self.img_size))

        # Normalize pixel values
        face_img = face_img.astype('float32') / 255.0

        # Reshape for model (add batch and channel dimensions)
        face_img = np.expand_dims(face_img, axis=-1)  # Add channel dimension
        face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension

        return face_img

    def predict_expression(self, face_img):
        """Predict facial expression"""
        predictions = self.expression_model.predict(face_img, verbose=0)
        predicted_class = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class]

        expression = self.class_names[predicted_class]

        return expression, confidence

    def draw_expression_box(self, frame, x, y, w, h, expression, confidence, color):
        """Draw enhanced bounding box with expression information"""
        # Draw main bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

        # Draw corner markers
        marker_size = 15
        cv2.line(frame, (x, y), (x + marker_size, y), color, 3)
        cv2.line(frame, (x, y), (x, y + marker_size), color, 3)
        cv2.line(frame, (x + w, y), (x + w - marker_size, y), color, 3)
        cv2.line(frame, (x + w, y), (x + w, y + marker_size), color, 3)
        cv2.line(frame, (x, y + h), (x + marker_size, y + h), color, 3)
        cv2.line(frame, (x, y + h), (x, y + h - marker_size), color, 3)
        cv2.line(frame, (x + w, y + h), (x + w - marker_size, y + h), color, 3)
        cv2.line(frame, (x + w, y + h), (x + w, y + h - marker_size), color, 3)

        # Prepare label text
        label_text = f"{expression} ({confidence:.2f})"

        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(
            label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
        )

        # Draw label background
        label_bg_y1 = y - text_height - 20
        label_bg_y2 = y
        cv2.rectangle(frame, (x, label_bg_y1),
                      (x + text_width + 20, label_bg_y2), color, -1)

        # Draw label text
        cv2.putText(frame, label_text, (x + 10, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Draw confidence bar
        bar_width = w
        bar_height = 10
        confidence_width = int(bar_width * confidence)
        cv2.rectangle(frame, (x, y + h + 5),
                      (x + bar_width, y + h + 5 + bar_height), (50, 50, 50), -1)
        cv2.rectangle(frame, (x, y + h + 5),
                      (x + confidence_width, y + h + 5 + bar_height), color, -1)


def main():
    print("🎭 REAL-TIME FACIAL EXPRESSION DETECTION")
    print("=" * 50)

    # Initialize detector
    detector = FacialExpressionDetector()

    if detector.expression_model is None:
        print("❌ Cannot start without trained model.")
        return

    # Initialize camera
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    if not cap.isOpened():
        print("❌ Error: Could not open camera")
        return

    print("✅ Camera initialized successfully!")
    print("🔍 Detecting expressions: Angry, Disgust, Fear, Happy, Neutral, Sad, Surprise")

    print("\n🎮 CONTROLS:")
    print("   Q - Quit application")
    print("   S - Save current frame")
    print("   C - Switch camera")

    frame_count = 0
    camera_index = 0
    prev_time = time.time()
    fps = 0

    # Expression statistics
    expression_stats = {expr: 0 for expr in detector.class_names}

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to grab frame")
                break

            # Calculate FPS
            current_time = time.time()
            fps = 1 / (current_time - prev_time)
            prev_time = current_time

            # Detect facial expressions
            processed_frame, expressions = detector.detect_expressions(frame)

            # Update statistics
            for expr_data in expressions:
                expression_stats[expr_data['expression']] += 1

            # Add information overlay
            cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(processed_frame, f"Faces: {len(expressions)}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Display current expressions
            y_offset = 90
            for i, expr_data in enumerate(expressions[:4]):  # Show first 4 faces
                expr_text = f"Face {i + 1}: {expr_data['expression']} ({expr_data['confidence']:.2f})"
                color = expr_data['color']
                cv2.putText(processed_frame, expr_text, (10, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                y_offset += 25

            # Display expression statistics
            stat_y = processed_frame.shape[0] - 150
            cv2.putText(processed_frame, "Expression Stats:", (10, stat_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            stat_y += 20

            # Show top 3 expressions
            sorted_stats = sorted(expression_stats.items(), key=lambda x: x[1], reverse=True)[:3]
            for expr, count in sorted_stats:
                if count > 0:
                    stat_text = f"{expr}: {count}"
                    color = detector.expression_colors.get(expr, (255, 255, 255))
                    cv2.putText(processed_frame, stat_text, (10, stat_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                    stat_y += 15

            # Add color legend
            legend_x = processed_frame.shape[1] - 200
            legend_y = 30
            cv2.putText(processed_frame, "Expression Colors:", (legend_x, legend_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            legend_y += 20

            for i, expr in enumerate(detector.class_names[:4]):  # Show first 4 in legend
                color = detector.expression_colors[expr]
                cv2.putText(processed_frame, expr, (legend_x, legend_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
                legend_y += 15

            # Add controls help
            cv2.putText(processed_frame, "Q:Quit  S:Save  C:Camera",
                        (10, processed_frame.shape[0] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

            # Display result
            cv2.imshow('Facial Expression Detection', processed_frame)

            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                # Save frame
                filename = f"expression_{frame_count:04d}.jpg"
                cv2.imwrite(filename, processed_frame)
                print(f"💾 Saved: {filename}")
                frame_count += 1
            elif key == ord('c'):
                # Switch camera
                camera_index = 1 - camera_index
                cap.release()
                cap = cv2.VideoCapture(camera_index)
                cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
                print(f"📷 Switched to camera {camera_index}")

    except KeyboardInterrupt:
        print("\n⏹️ Stopped by user")

    finally:
        # Print final statistics
        print("\n📊 Final Expression Statistics:")
        total_detections = sum(expression_stats.values())
        for expr, count in sorted(expression_stats.items(), key=lambda x: x[1], reverse=True):
            if count > 0:
                percentage = (count / total_detections) * 100
                print(f"   {expr}: {count} ({percentage:.1f}%)")

        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Expression detection stopped. Goodbye!")


if __name__ == "__main__":
    main()

🎭 REAL-TIME FACIAL EXPRESSION DETECTION


✅ Facial expression model loaded successfully!
✅ Camera initialized successfully!
🔍 Detecting expressions: Angry, Disgust, Fear, Happy, Neutral, Sad, Surprise

🎮 CONTROLS:
   Q - Quit application
   S - Save current frame
   C - Switch camera

📊 Final Expression Statistics:
   happy: 452 (100.0%)
✅ Expression detection stopped. Goodbye!
